## ENHANCING STORE SALES PREDICTION FOR FAVORITA.

## 1. BUSINESS UNDERSTANDING
Corporation Favorita is a significant actor in Ecuador's dynamic economic environment and contributes to the country's economic diversification as a result of the country's strong reliance on oil prices. Favorita, a complex Ecuadorian business, operates both inside and outside of national borders and makes investments in a range of industries to support economic resilience.

Corporation Favorita is aware of the crucial necessity for well-informed decision-making in its pursuit of sustainable growth and economic stability. In order to achieve this, the company aims to maximize the potential of data-driven insights, particularly with regard to its grocery retail operations. Favorita intends to bolster its market presence, optimize its offers, and contribute to the resiliency of the country's economy by diving into the complex patterns of retail sales, product dynamics, and transaction behaviors.

## Objectives of the Project.
1.  Investigate the performance variations across different Favorita stores.
2. Create reliable time series forecasting models that can accurately anticipate unit sales for the wide range of products sold in different Favorita stores.
3. Assess the impact of promotional strategies on sales. By analyzing the correlation between promotions and sales spikes, the project aims to determine the efficacy of different promotional tactics and optimize their deployment.
4. Conduct a comprehensive analysis of annual transaction volumes, trends, and variations.

## 2. DATA COLLECTION AND DATA PREPARATION

    2.1 DATA COLLECTION

Installations and Importing the necessary libraries

In [1]:
#Install pyodbc and python-dotenv
%pip install pyodbc  
%pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [2]:
#Importing the necessary Libraries.
# Libraries to collect and load/read data
import pyodbc                    #
from dotenv import dotenv_values #import the dotenv_values function from the dotenv package
import pandas as pd
import numpy as np
import os
import warnings 

#Libraries for Vizualization and EDA
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns